In [2]:
pip install opencv-contrib-python==4.6.0.66

In [ ]:
import cv2
import cv2.aruco as aruco
from google.colab.patches import cv2_imshow

cap = cv2.VideoCapture('/content/ArUco_marker.mp4')
#原始畫面有點大，為了有利於顯示這份講義所以縮小。
frameWidth   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//2
frameHeight  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//2

arucoParams = aruco.DetectorParameters_create()
arucoDict   = aruco.Dictionary_get(aruco.DICT_7X7_50)

print('height {}, width {}'.format(cap.get(cv2.CAP_PROP_FRAME_HEIGHT),cap.get(cv2.CAP_PROP_FRAME_WIDTH)))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame,(frameWidth,frameHeight))
    (corners, ids, rejected) = aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)
    if len(corners) > 0:
        aruco.drawDetectedMarkers(frame, corners, ids)

    #cv2_imshow(frame)
    if cv2.waitKey(20) != -1:
        break

cv2.destroyAllWindows()
cap.release()

height 1080.0, width 1920.0


In [4]:
import cv2
import cv2.aruco as aruco

cap = cv2.VideoCapture('/content/ArUco_marker.mp4')
#原始畫面有點大，為了有利於顯示這份講義所以縮小。
totalFrame   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//2
frameHeight  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//2

arucoParams  = aruco.DetectorParameters_create()
arucoParams.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX

arucoDict    = aruco.Dictionary_get(aruco.DICT_7X7_50)

# 必須描述ChArUco board的尺寸規格
gridX        = 5 # 水平方向5格
gridY        = 7 # 垂直方向7格
squareSize   = 4 # 每格為4cmX4cm
# ArUco marker為2cmX2cm
charucoBoard = aruco.CharucoBoard_create(gridX,gridY,squareSize,squareSize/2,arucoDict)

print('height {}, width {}'.format(cap.get(cv2.CAP_PROP_FRAME_HEIGHT),cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
refinedStrategy = True
criteria        = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.00001)
frameId        = 0
collectCorners = []
collectIds     = []
collectFrames  = []
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame,(frameWidth,frameHeight))
    (corners, ids, rejected) = aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)

    if refinedStrategy:
        corners, ids, _, _ = aruco.refineDetectedMarkers(frame,charucoBoard,corners,ids,rejected)

    if frameId % 100 == 50 and ids is not None and len(ids)==6: # 17 ArUco markers
        collectCorners.append(corners)
        collectIds.append(ids.ravel())
        collectFrames.append(frame)

    if len(corners) > 0:
        aruco.drawDetectedMarkers(frame, corners, ids)

    #cv2_imshow(frame)
    if cv2.waitKey(20) != -1:
        break

    frameId += 1

cv2.destroyAllWindows()
cap.release()
print(collectCorners)

height 1080.0, width 1920.0
[(array([[[568.9986 , 481.28015],
        [441.8235 , 454.118  ],
        [468.63052, 329.9016 ],
        [593.0125 , 353.31848]]], dtype=float32), array([[[364.9724 , 438.07312],
        [245.93362, 412.3922 ],
        [276.1745 , 292.90613],
        [393.35333, 315.6897 ]]], dtype=float32), array([[[399.96432, 286.03528],
        [283.36633, 263.7638 ],
        [312.40363, 151.78868],
        [426.62256, 171.4932 ]]], dtype=float32), array([[[432.76135 , 143.5785  ],
        [319.14252 , 124.43482 ],
        [346.26584 ,  18.490389],
        [457.7556  ,  35.30743 ]]], dtype=float32), array([[[598.6465 , 322.35233],
        [474.7926 , 299.73297],
        [499.9739 , 183.25932],
        [621.1297 , 202.70097]]], dtype=float32), array([[[626.35284, 173.80681],
        [505.926  , 154.93411],
        [529.3411 ,  45.25749],
        [647.39246,  61.20594]]], dtype=float32)), (array([[[556.58374, 310.5354 ],
        [439.1636 , 291.11343],
        [464.35574, 

In [5]:
import numpy as np
caliCorners=np.concatenate([np.array(x).reshape(-1,4,2) for x in collectCorners],axis=0)
counter=np.array([len(x) for x in collectIds])
caliIds=np.array(collectIds).ravel()
cameraMatrixInit = np.array([[ 1000.,    0., frameWidth/2.],[    0., 1000., frameHeight/2.],[    0.,    0.,           1.]])
distCoeffsInit   = np.zeros((5,1))
ret, aruco_cameraMatrix, aruco_distCoeffs, aruco_rvects, aruco_tvects = aruco.calibrateCameraAruco(caliCorners,caliIds,counter,charucoBoard,(frameWidth,frameHeight),cameraMatrixInit,distCoeffsInit)
print(aruco_cameraMatrix)
print(aruco_distCoeffs)
print(aruco_rvects)
print(aruco_tvects)

[[ 72.17616465   0.         521.68478354]
 [  0.         215.61849698 265.29022944]
 [  0.           0.           1.        ]]
[[-5.70206150e-03]
 [-5.37763257e-05]
 [ 1.64137542e-03]
 [ 3.90645812e-03]
 [ 2.23724267e-07]]
(array([[1.17733085],
       [1.45050383],
       [2.93886401]]), array([[ 1.741927  ],
       [-0.60649517],
       [ 1.17468314]]), array([[ 1.57302593],
       [-1.07073419],
       [ 1.02900863]]), array([[1.34984937],
       [1.38238217],
       [2.82530971]]), array([[ 1.73440828],
       [-0.47552128],
       [ 0.98614277]]))
(array([[ 3.17496123],
       [ 3.29124999],
       [-6.65369239]]), array([[10.01723327],
       [ 2.17250526],
       [-7.68330296]]), array([[  1.43971733],
       [  3.01375554],
       [-12.34239854]]), array([[-2.61725886],
       [ 3.31259834],
       [-4.67664213]]), array([[ 7.69511743],
       [ 0.83404069],
       [-8.79940866]]))


In [18]:
import cv2
import cv2.aruco as aruco
from google.colab.patches import cv2_imshow

cap = cv2.VideoCapture('/content/ArUco_marker.mp4')
cap1 = cv2.VideoCapture('/content/bean.mp4')

if not all([cap.isOpened(), cap1.isOpened()]):
    print("Error opening video files!")
    exit()

#原始畫面有點大，為了有利於顯示這份講義所以縮小。
totalFrame   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//2
frameHeight  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//2

arucoParams  = aruco.DetectorParameters_create()
arucoParams.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX

arucoDict    = aruco.Dictionary_get(aruco.DICT_7X7_50)

# 必須描述ChArUco board的尺寸規格
gridX        = 2 # 水平方向5格
gridY        = 3 # 垂直方向7格
squareSize   = 4 # 每格為4cmX4cm
# ArUco marker為2cmX2cm
charucoBoard = aruco.CharucoBoard_create(gridX,gridY,squareSize,squareSize/2,arucoDict)

print('height {}, width {}'.format(cap.get(cv2.CAP_PROP_FRAME_HEIGHT),cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
refinedStrategy = True
criteria        = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.00001)
frameId        = 0
collectCorners = []
collectIds     = []
collectFrames  = []

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('output.mp4',fourcc,20.0,(frameWidth,frameHeight))


while True:
    ret, frame = cap.read()
    ret1, frame1 = cap1.read()

    if not all([ret,ret1]):
        break

    print(frame1.shape)
    print("--")

    frame = cv2.resize(frame,(frameWidth,frameHeight))
    (corners, ids, rejected) = aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)

    #print(corners)

    h1,w1,dim = frame1.shape
    pst_src = np.array(
        [
            [w1 - 1, h1 -1],
            [0, h1 -1],
            [0,0],
            [w1 - 1,0],
        ],dtype = float
    )
    print("pst_src",pst_src)
    points_2d = np.vstack(corners[0]).astype(float)

    print("corners[0] =",corners[0])
    print("points_2d =", points_2d)
    h1, status = cv2.findHomography(pst_src,points_2d)

    print("h1 =",h1)
    print(frame1.shape)

    print("%%")

    frame1 = cv2.warpPerspective(frame1,h1,(frame.shape[1],frame.shape[0]))

    print(frame1.shape)
    print("---")
    cv2.fillConvexPoly(frame,corners[0].astype(int),0,16)

    print(frame.shape)
    print("---")
    print(frame1.shape)
    frame = cv2.add(frame,frame1)

    if refinedStrategy:
        corners, ids, _, _ = aruco.refineDetectedMarkers(frame,charucoBoard,corners,ids,rejected)

    if frameId % 100 == 50 and ids is not None and len(ids)==6: # 6 ArUco markers
        collectCorners.append(corners)
        collectIds.append(ids.ravel())
        collectFrames.append(frame)

    if len(corners) > 0:
        aruco.drawDetectedMarkers(frame, corners, ids)

    #cv2_imshow(frame)
    out.write(frame)

    frameId += 1

cv2.destroyAllWindows()
cap.release()
cap1.release()
#print(collectCorners)

Streaming output truncated to the last 5000 lines.
 [-2.39616028e-05 -3.78210946e-05  1.00000000e+00]]
(720, 1280, 3)
%%
(540, 960, 3)
---
(540, 960, 3)
---
(540, 960, 3)
(720, 1280, 3)
--
pst_src [[1279.  719.]
 [   0.  719.]
 [   0.    0.]
 [1279.    0.]]
corners[0] = [[[626.4399  494.07697]
  [497.50696 469.0867 ]
  [519.8736  343.58655]
  [645.7574  363.8354 ]]]
points_2d = [[626.43988037 494.07696533]
 [497.50695801 469.08670044]
 [519.87359619 343.58654785]
 [645.75738525 363.83538818]]
h1 = [[ 8.27837291e-02 -5.06831129e-02  5.19873596e+02]
 [ 7.01989009e-03  1.56091300e-01  3.43586548e+02]
 [-2.42194280e-05 -3.93464490e-05  1.00000000e+00]]
(720, 1280, 3)
%%
(540, 960, 3)
---
(540, 960, 3)
---
(540, 960, 3)
(720, 1280, 3)
--
pst_src [[1279.  719.]
 [   0.  719.]
 [   0.    0.]
 [1279.    0.]]
corners[0] = [[[632.3576  493.78647]
  [502.92276 468.94504]
  [525.23193 343.39862]
  [651.6621  363.60956]]]
points_2d = [[632.35760498 493.78646851]
 [502.92276001 468.94503784]
 [525.2